# Decision trees for classification task with information gain as a splitting-criterion

#### Information Gain as splitting criterion

The entropy with respect to the target class variable $y$ of a training data set $\mathcal D$ is defined as:

$$
 H(y, \mathcal D) = - \sum_{y \in \mathcal Y} p(y|\mathcal D) \log_2 p(y|\mathcal D)
$$
with the domain of the target values $\mathcal Y = \{t_1, t_2,... \}$.


The probabilities are estimated by 
$$
  p(y=t_i, \mathcal D) = |\mathcal D^{(y=t_i)}| /|\mathcal D| 
$$    


with the number of training data $|\mathcal D|$  and the number of training data $|\mathcal D^{(y=t_i)}|$ with target label $t_i$: 


On a node a (binary) split on a feature $x_k$ is made by the split rule $x_k \leq v$. 
As result there are two data sets $\mathcal D_0$ and $\mathcal D_1$ for the left resp. the right branch.

The feature $x_k$ and the split value $v$ are choosen that they maximize the 'reduction of the entropy' measured by the information gain $I$:
$$
  I(y; x_k) = H(y, \mathcal D) - H(y|x_k) = H(y, \mathcal D) - \sum_{j=0}^1 p_jH(y, \mathcal D_j) =
  H(y, \mathcal D) + \sum_{j=0}^1  \sum_{y \in \mathcal Y} \frac{|\mathcal D_j|}{|\mathcal D|} p(y|\mathcal D_j) \log_2 p(y|\mathcal D_j)
$$
Note that $p_{j=0}$  is the estimated probability that a random data record of $\mathcal D$ has feature value $x_k \leq v$ which can be estimated by ${|\mathcal D_0|}/{|\mathcal D|}$ (analog for $j=1$).

$p(y=t_i|\mathcal D_0)$ can also be estimated by the fraction of the counts ${|\mathcal D_0^{(y=t_i)}|}/{|\mathcal D_0|}$. 
So the information gain can be computed just with counts:


$$
  I(y; x_k) = 
   \sum_{y \in \mathcal Y} \frac{|\mathcal D^{(y=t_i)}|}{|\mathcal D|}  \log_2 \frac{|\mathcal D^{(y=t_i)}|}{|\mathcal D|} + \sum_{j=0}^1  \sum_{y \in \mathcal Y} \frac{|\mathcal D_j^{(y=t_i)}|}{|\mathcal D|}  \log_2 \frac{|\mathcal D_j^{(y=t_i)}|}{|\mathcal D_j|}
$$


<!--$|\mathcal D_0|$ respectivly $|\mathcal D_1|$ is the number of elements in the splitted data sets.-->
 

#### Overfitting

Deep decision trees generalize often poorly. The following remedies reduce overfitting: 

- Limitation of the maximal depth of the tree. 
- Pruning with an validation set either during training (pre-pruning) or after training (post-pruning).
- Dimensionality reduction (reducing the number of features before training)


Also often combining decision trees to an ensemble (decision forests) is used against overfitting.  

### Example: Survival of the Titanic 


First you have read in the titanic data with pandas:

In [1]:
import numpy as np
import scipy as sc
import pandas as pd; print("pandas version: ", pd.__version__)

import sklearn as sk; print("scikit version: ", sk.__version__)
from sklearn.preprocessing import Imputer
from sklearn import tree
from sklearn.externals.six import StringIO

from os import system

import pandas as pd

from collections import Counter
from itertools import chain


pandas version:  0.24.2


ModuleNotFoundError: No module named 'sklearn'

`train_df` is a [_pandas_](http://pandas.pydata.org/) [data frame](http://pandas.pydata.org/pandas-docs/stable/dsintro.html). 
Let's view the data. 

In [ ]:
train_df = pd.read_csv("titanic-train.csv")
train_df.head(7)

Scikit's learn decision trees can handle only numeric data. Please convert the nominal `Sex` feature. 

In [ ]:
train_df["Sex"] = train_df['Sex'].map({'female': 1, 'male': 0})
train_df.head(7)

`Survived` is the target, that we want to predict from the values of the other columns.   
But not all of the other columns are helpful for classification. So we choose a feature set by hand and convert the features into a numpy array for scikit learn. 

In [ ]:
#column stack - Take a sequence of 1-D arrays and stack them as columns to make a single 2-D array. 

y = targets = labels  =  train_df["Survived"]
columns = ["Fare", "Pclass", "Sex", "Age", "SibSp"]
features = np.column_stack([train_df["Fare"], train_df["Pclass"], train_df["Sex"],train_df["Age"],train_df["SibSp"]])
features

There are missing values (`nan`). Please use the scikit learn `Imputer` to replace them by the mean of the columns.

In [ ]:
from sklearn.preprocessing import Imputer

imputer = Imputer(missing_values='NaN', strategy = 'mean', axis=0)
imputer = imputer.fit(features)
features = imputer.transform(features)
features

Now we are ready to learn a decision tree by the criterion 'Information Gain' and we restrict the depth of the tree to 3.
We use the [scikit learn decison tree module](http://scikit-learn.org/stable/modules/tree.html).

In [ ]:
from sklearn import tree

clf = tree.DecisionTreeClassifier(criterion="entropy", max_depth=3)
clf = clf.fit(features, y)

`clf` is an instance of a trained decision tree classifier.

The decision tree can be visualized. For this we must write an graphviz dot-File  

In [ ]:
from sklearn.externals.six import StringIO

with open("titanic.dot", 'w') as f:
    f = tree.export_graphviz(clf, out_file=f, feature_names=columns)

The dot file can be converted with the graphiz `dot`- renderer to an image.

    dot -Tpng titanic.dot -o titanic.png

Here is the graph:

<img src="images/titanic.png" width="1000px"/>


According to the decision tree the main criterion (root node) for survival is the sex of the passenger. In the left subtree are the male passengers (sex = 0), in the right subtree the female (sex=1). 

In the leafs the class information is given by a `value` array. Here the second value is the number of survivers in the leafs.

For example the leftmost leaf represents passengers that are male (sex=0) with fare<=26.2687 and age<=13.5. 13 of such boys survived and 2 of them died.

The entropy $- \sum p_i \log_2 (p_i)$ is displayed also at each node (splitting criterion).


### Exercises: Splitting criterion entropy / information gain

#### Exercise 1: 

Compute the root node entropy (with numpy).

$$
Entropy \space H = \sum_{i}[p_{i} * I_{i}] = - \sum_{i}[p_{i} * log_{2} p_{i}]
$$

In [ ]:
def entropy(labels):
    counter = Counter(labels)
    i = np.array(list(chain(counter.values())))
    p = i / len(labels)
    return - (p * np.log2(p)).sum()

entropy(y)

#### Exercise 2

Compute the information gain of the first split node (root node). Use the entropy values and the number of data records (samples)
from the decision tree image. 




In [ ]:
records = y.size
records_left = train_df[train_df['Sex'] == 0].shape[0]
records_right = train_df[train_df['Sex'] == 1].shape[0]
entropy_first_node = entropy(y)
entropy_left = 0.699
entropy_right = 0.824
entropy_kids = (records_left / records) * entropy_left + (records_right / records) * entropy_right
info_gain = entropy_first_node - entropy_kids
info_gain

#### Exercise 3
Compute the information gain of the following split table:

|  | class 0  | class 1  |  
|---|---|---|
| feature <= v| 2  | 13  |      
| feature > v | 359  |  41 |   

The numbers are the corresponding data records, e.g. there are 13 data records with target class 1 and feature <= v. 

Write a python function that computes the information gain.The data is given by a python array:

In [ ]:
data = np.array([[2.,13.],[359., 41.]])

def info_gain(data):
    records = data.sum()
    
    records_left = data[0].sum()
    p_left = data[0] / records
    entropy_left = - (p_left * np.log2(p_left)).sum()
    print(entropy_left)
    
    records_right = data[1].sum()
    p_right = data[1] / records
    entropy_right = - (p_right * np.log2(p_right)).sum()
    
    entropy_kids = (records_left / records) * entropy_left + (records_right / records) * entropy_right
    
    p_initial = np.array([data[:,0].sum(),data[:,1].sum()]) / records
    entropy_initial = - (p_initial * np.log2(p_initial)).sum()
    
    info_gain = entropy_initial - entropy_kids
    return info_gain
    
    
info_gain(data)